[原文链接](https://ar5iv.labs.arxiv.org/html/1810.04805?_immersive_translate_auto_translate=1)

## 引言 Intro

## 相关工作 Related Work

### 无监督的基于特征的方法 Unsupervised Feature-based Approaches

## 模型架构 BERT

本节介绍 BERT 及其详细实现。我们的框架分为两个步骤： 预培训和微调 。

- 在预训练过程中，模型会在不同的预训练任务中对未标记的数据进行训练。
- 在微调中，首先用预训练参数初始化 BERT 模型，然后利用下游任务的标记数据对所有参数进行微调。每个下游任务都有独立的微调模型，尽管它们初始化时使用相同的预训练参数。[图 1](#fig_1) 中的问答示例将作为本节的持续示例。
- BERT 的一个显着特点是其跨不同任务的统一架构。预训练架构和最终下游架构之间的差异很小。

<a id="fig_1"></a>
<div style="width: 80%; background-color: white; margin: 0 auto; padding: 20px; justify-content: center; text-align: center;">
    <img src="./assets/BERT_Overall.png" />
    <span style="font-size: 0.8em; color: gray;">图 1：BERT 的总体预训练和微调程序。除了输出层之外，预训练和微调都使用相同的架构。相同的预训练模型参数用于初始化不同下游任务的模型。在微调过程中，所有参数都会进行微调。[CLS] 是添加到每个输入示例前面的特殊符号，[SEP] 是特殊的分隔符标记（例如分隔问题/答案）。</span>
</div>

$\text{BERT}_{\text{BASE}}$ 被选择为与 OpenAI GPT 具有相同的模型大小以进行比较。然而，关键是 BERT Transformer 使用双向自注意力，而 GPT Transformer 使用受限的自注意力，其中每个标记只能关注其左侧的上下文。

- $\text{BERT}_{\text{BASE}}$: (L = 12, H = 768, A = 12, 总共参数 = 110M):
  - $L$: 层数
  - $H$: 隐含层大小 
  - $A$: 注意力头数

__输入/输出表示 Input/Output Representations__

- 为了让BERT处理各种下游任务，我们的输入表示能够在一个标记序列中明确地表示单个句子和一对句子。（例如\<Question, Answer\>）
- Sentence: 可以是连续文本的任意跨度，而不是实际的语言句子。
- Sequence: 是指输入给BERT的标记序列，它可以是单个句子或两个句子打包在一起。

- 使用WordPiece嵌入和30,000个标记词汇表。
- 每个句子的第一个标记是特殊的分类标记`[CLS]`。
  - 最后一层隐含层`[CLS]`位置的状态用来表示整个句子的聚合表示(用于分类任务) - The final hidden state corresponding to this token is used as the aggregate sequence representation for classification tasks.
- 两种方式来区分句子A和句子B:
  - 通过在每个标记的嵌入中添加一个句子A或句子B的段嵌入($E_A$或$E_B$)。
  - 通过在句子之间插入特殊的分隔符标记`[SEP]`。
- 最后一层的:
  - `[CLS]`: $C \in \mathbb{R}^{H}$
  - 第$i$个标记: $T_i \in \mathbb{R}^{H}$

<div style="width: 80%; background-color: white; margin: 0 auto; padding: 20px; justify-content: center; text-align: center;">
    <img src="./assets/Input_Emebeddings.png"/>
    <span style="font-size: 0.8em; color: gray;">图 2：BERT 输入表示。输入嵌入是标记嵌入、分段嵌入和位置嵌入的总和。</span>
</div>

<!-- Figure 2:BERT input representation. The input embeddings are the sum of the token embeddings, the segmentation embeddings and the position embeddings. -->

### 预训练任务 Pre-training Tasks

- 与之前的研究不同，我们不使用传统的从左->右或者从右->左的语言模型预训练目标。
- 相反，我们使用两种新的务监督任务，如[图 1-1](#fig_1_1)所示:
  - 掩码语言模型(MLM)
  - 下一句预测(NSP)

<a id="fig_1_1"></a>
<div style="width: 50%; /* 限制容器宽度为50% */ background-color: white; margin: 0 auto; padding: 20px; justify-content: center; text-align: center; overflow: hidden; /* 隐藏超出容器的内容（右侧） */">
    <img src="./assets/BERT_Overall.png" style="position: relative; left: 0; /* 确保图片从左侧开始显示 */ max-width: 230%; /* 让图片宽度是容器的2倍，保证只显示左半部分 */ height: auto; /* 保持宽高比 */" />
    <span style="font-size: 0.8em; color: gray;">图 1-1：模型预训练部分</span>
</div>

__Task #1: 掩码语言模型(Masked LM)__

- 随机屏蔽每个序列中所有WordPiece token的`15%`:
- $[[CLS], Tok1, Tok2 , ..., TokN, [SEP], Tok1', Tok2', ..., TokM', [SEP]]$
  - 其中`15%`的标记被替换为:
    - `80%`的时间: `[MASK]`标记
    - `10%`的时间: 随机标记
    - `10%`的时间: 保持不变
- 如果第$i$个位置被替换，那么需要预测原始的标记:
- 通过embedding层传递修改后的序列:
- $[[E_{[CLS]}], [E_{Tok1}], [E_{Tok2}] , ..., [E_{TokN}], [E_{[SEP]}], [E_{Tok1'}], [E_{Tok2'}], ..., [E_{TokM'}], [E_{[SEP]}]]$
- 通过位置和段嵌入添加位置信息和句子信息:
- $E_{i} = E_{Tok_i} + E_{Pos_i} + E_{Seg_i}$  -> + 位置嵌入 + 段嵌入
  - 每一个$E_{i} \in \mathbb{R}^{H}$, 比如具体的$H=768$, 那就是一个嵌入就是768维的向量
- 通过BERT:
  - 前馈层: 每个$E_{i} \overset{线性1(W_1 + B_1)}{\longrightarrow} H_{i} \overset{ReLU + 线性2(W_2 + B_2)}{\longrightarrow} E_{i}'$
    - 中间层$H_{i} \in \mathbb{R}^{4H}$, 比如具体的$4H=3072$,论文中BERT-Base/Large都是3072
    - $W_{1/2} \in \mathbb{R}^{768 \times 3072}$, $B_{1/2} \in \mathbb{R}^{3072}$
- 最后一层输出:
- $[C, T_1, T_2, ..., T_N, T_{[SEP]}, T_1', T_2', ..., T_M', T_{[SEP]}']$
- $T_i \overset{投影层 (W_{proj} + B_{proj})}{\longrightarrow} L_i \overset{Softmax}{\longrightarrow} P_i$
  - $V$ 是词汇表大小, 例如30522
  - $W_{proj}, \in \mathbb{R}^{768 \times 30522}, B_{proj} \in \mathbb{R}^{30522}$
  - $L_i \in \mathbb{R}^{30522}$ 是预测的logits
  - $P_i \in \mathbb{R}^{30522}$ 是预测的概率分布


__Task #2: 下一句预测(Next Sentence Prediction)__


许多重要的下游任务，如问答（QA）和自然语言推理（NLI），都是基于理解两个句子之间的关系。
- 投影层: $C \overset{线性(W_{NSP} + B_{NSP})}{\longrightarrow} L_{NSP} \overset{Softmax}{\longrightarrow} P_{NSP}$
  - $W_{NSP} \in \mathbb{R}^{768 \times 2}$, $B_{NSP} \in \mathbb{R}^{2}$
  - $L_{NSP} \in \mathbb{R}^{2}$ logits
  - $P_{NSP} \in \mathbb{R}^{2}$ [P(isNext), P(NotNext)]
- `[CLS]`位置的最终隐藏状态$C$被用来预测句子B是否是句子A的下一个句子。

__预训练数据集 Pre-training Data__

- BooksCorpus(8亿字)
- English Wikipedia(25亿字)

### 微调BERT Fine-tuning BERT

- 文本对任务需要**两个句子**才能完成任务：
  1. 传统方法的麻烦之处：
     1) 独热编码。把2个句子分开输入模型，各自生成表示（A理解A，B理解B，互不干扰）
     2) 双向交叉注意力。单独设计一个交叉注意力模块，是让A和B的表示相互看（比如让问题B的每个词去匹配文章A中的相关词）
  2. BERT简化思路：用"自注意力"把两步合成一步
     1) 拼接文本对: [CLS] A [SEP] B [SEP]
     2) 用**自注意力**统一编码。自注意力可以让序列中的每个token都去看其他所有的词。

- 对于每个任务，我们只需将任务特定的输入和输出输入 BERT，并对所有参数进行端到端的微调。 在输入端，预训练中的句子 A 和句子 B 类似于:
  1. 文本蕴涵（entailment）：两个句子是否相近: [CLS] 句子A [SEP] 句子B [SEP], 输出用[CLS]接分类层。
  2. 文本蕴涵（entailment）：两个句子是否蕴含: [CLS] 前提句 [SEP] 假设句 [SEP], 输出用[CLS]接分类层。
  3. 问答（Question Answering）：[CLS] 问题 [SEP] 文章 [SEP], 两个输出层：一个预测答案开始位置，一个预测答案结束位置
  4. 单文本任务（如分类、序列标记）： 文本分类：判断句子情感；序列标记：给每个词标 “实体”。分类：用[CLS]接分类层。序列：$T_i$接序列标记层。
- 在输出端，token表示被输入到一个输出层，用于token级任务，如序列标记或问答，而 [CLS] 表示则被输入到输出层进行分类，如蕴涵或情感分析。

<a id="fig_1_1"></a>
<div style="width: 50%; background-color: white; margin: 0 auto; padding: 20px; justify-content: center; text-align: center; overflow: hidden; ">
    <div style="width: 100%; overflow: hidden; display: inline-block;">
        <img src="./assets/BERT_Overall.png" style="position: relative; left: -96%; max-width: 200%; height: auto;" />
    </div>
    <span style="font-size: 0.8em; color: gray;">图 1-2：BERT微调部分</span>
</div>


| System               | MNLI-(m/mm) | QQP  | QNLI | SST-2 | CoLA | STS-B | MRPC | RTE  | Average |
|----------------------|-------------|------|------|-------|------|-------|------|------|---------|
|                      | 392k        | 363k | 108k | 67k   | 8.5k | 5.7k  | 3.5k | 2.5k | --      |
| **Pre-OpenAI SOTA**  | 80.6/80.1   | 66.1 | 82.3 | 93.2  | 35.0 | 81.0  | 86.0 | 61.7 | 74.0    |
| **BiLSTM+ELMo+Attn** | 76.4/76.1   | 64.8 | 79.8 | 90.4  | 36.0 | 73.3  | 84.9 | 56.8 | 71.0    |
| **OpenAI GPT**       | 82.1/81.4   | 70.3 | 87.4 | 91.3  | 45.4 | 80.0  | 82.3 | 56.0 | 75.1    |
| **BERT<sub>BASE</sub>** | 84.6/83.4 | 71.2 | 90.5 | 93.5  | 52.1 | 85.8  | 88.9 | 66.4 | 79.6    |
| **BERT<sub>LARGE</sub>** | 86.7/85.9 | 72.1 | 92.7 | 94.9  | 60.5 | 86.5  | 89.3 | 70.1 | 82.1    |

#### 更多细节

<div style="width: 80%; background-color: white; margin: 0 auto; padding: 20px; justify-content: center; text-align: center;">
    <img src="./assets/Bert_comparisons.png"/>
    <span style="font-size: 0.8em; color: gray;">图 3： 预训练模型架构的差异。BERT 使用双向 Transformer。OpenAI GPT 使用从左到右的 Transformer。ELMo 使用独立训练的从左到右和从右到左的 LSTM 的串联来生成下游任务的特征。在这三者中，只有 BERT 表示在所有层中共同以左右上下文为条件。除了架构差异之外，BERT 和 OpenAI GPT 是微调方法，而 ELMo 是一种基于功能的方法。</span>
</div>


<!-- Figure 3:Differences in pre-training model architectures. BERT uses a bidirectional Transformer. OpenAI GPT uses a left-to-right Transformer. ELMo uses the concatenation of independently trained left-to-right and right-to-left LSTMs to generate features for downstream tasks. Among the three, only BERT representations are jointly conditioned on both left and right context in all layers. In addition to the architecture differences, BERT and OpenAI GPT are fine-tuning approaches, while ELMo is a feature-based approach. -->
<!-- - 图 3： 预训练模型架构的差异。BERT 使用双向 Transformer。OpenAI GPT 使用从左到右的 Transformer。ELMo 使用独立训练的从左到右和从右到左的 LSTM 的串联来生成下游任务的特征。在这三者中，只有 BERT 表示在所有层中共同以左右上下文为条件。除了架构差异之外，BERT 和 OpenAI GPT 是微调方法，而 ELMo 是一种基于功能的方法。 -->

<div style="width: 60%; background-color: white; margin: 0 auto; padding: 20px; display: flex; justify-content: center;">
    <img src="./assets/Bert_fine_tune.png"/>
</div>